#### There are several duplicative pairs (*e.g.* index 2322 vs. 34007) in the pairs.csv as well as other binary datasets created by nearest-neighbor searches.
#### Such pairs can cause leakage during CV.
#### Herein, one of the ways to tackle with the problem is shown.

In [ ]:
import pandas as pd 
from tqdm.notebook import tqdm
tqdm.pandas()
pairs = pd.read_csv("../input/foursquare-location-matching/pairs.csv", nrows=40000).reset_index(drop=True)

print(pairs.loc[2322])
print(pairs.loc[34007])

In [ ]:
def drop_duplicate(df):
    del_count=0
    check_id = df["id_1"].values.tolist()
    check_match_id = df["id_2"].values.tolist()
    del_idx_list = []
    for idx in tqdm(reversed(range(len(check_id)))):
        id_list = [i for i, val in enumerate(check_match_id) if val == check_id[idx]]
        match_id_list = [i for i, val in enumerate(check_id) if val == check_match_id[idx]]
        dup_list = list(set(id_list)&set(match_id_list))
        if len(dup_list) != 0:
            del_idx_list.append(idx)
            del_count+=1
        del check_id[idx], check_match_id[idx]
    print("del_count:", del_count)
    df.drop(df.index[del_idx_list], inplace=True)
    return df

pairs_modified = drop_duplicate(pairs)

### The duplicative row in the pair (index 34007) has been deleted.

In [ ]:
print(pairs_modified.loc[2322])
try:
    print(pairs_modified.loc[34007])
except:
    print("\nCannot find the row.")

### Grouping the data by country can save processing time.

In [ ]:
def drop_duplicate(df):
    del_count=0
    check_id = df["id_1"].values.tolist()
    check_match_id = df["id_2"].values.tolist()
    del_idx_list = []
    for idx in reversed(range(len(check_id))):
        id_list = [i for i, val in enumerate(check_match_id) if val == check_id[idx]]
        match_id_list = [i for i, val in enumerate(check_id) if val == check_match_id[idx]]
        dup_list = list(set(id_list)&set(match_id_list))
        if len(dup_list) != 0:
            del_idx_list.append(idx)
            del_count+=1
        del check_id[idx], check_match_id[idx]
    df.drop(df.index[del_idx_list], inplace=True)
    return df, del_count

pairs = pd.read_csv("../input/foursquare-location-matching/pairs.csv", nrows=40000).reset_index(drop=True)
all_df = []
del_count_all=0
for country, country_df in tqdm(pairs.groupby("country_1", as_index=False)):
    tmp_df, del_count = drop_duplicate(country_df)
    all_df.append(tmp_df)
    del_count_all += del_count
pairs_modified_2 = pd.concat(all_df)
print("del_count:", del_count_all)

print(pairs_modified_2.loc[2322])
try:
    print(pairs_modified_2.loc[34007])
except:
    print("\nCannot find the row.")